In [1]:
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer  
from llm_foundry.scripts.train.dataloader import *

model_path = "vinai/PhoGPT-7B5-Instruct"  
  
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)  
config.init_device = "cuda"
  
model = AutoModelForCausalLM.from_pretrained(  
    model_path, config=config, torch_dtype=torch.bfloat16, trust_remote_code=True  
)

model.eval()  
  
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)  
  
"""PROMPT = "### Câu hỏi:\n{instruction}\n\n### Trả lời:"  
  
input_prompt = PROMPT.format_map(  
    {"instruction": "Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính"}  
)  
  
input_ids = tokenizer(input_prompt, return_tensors="pt")  
  
outputs = model.generate(  
    inputs=input_ids["input_ids"].to("cuda"),  
    attention_mask=input_ids["attention_mask"].to("cuda"),  
    do_sample=True,  
    temperature=1.0,  
    top_k=50,  
    top_p=0.9,  
    max_new_tokens=1024,  
    eos_token_id=tokenizer.eos_token_id,  
    pad_token_id=tokenizer.pad_token_id  
)  
  
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  
response = response.split("### Trả lời:")[1]"""

/usr/lib/python3/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-7B5-Instruct/d1a5a418bf01d49e8bf1b5b737b8ef143a33d9fd/configuration_mpt.py:97: UserWarning: alibi is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi is turned on, setting `learned_pos_emb` to `False.`')
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.97s/it]


'PROMPT = "### Câu hỏi:\n{instruction}\n\n### Trả lời:"  \n  \ninput_prompt = PROMPT.format_map(  \n    {"instruction": "Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính"}  \n)  \n  \ninput_ids = tokenizer(input_prompt, return_tensors="pt")  \n  \noutputs = model.generate(  \n    inputs=input_ids["input_ids"].to("cuda"),  \n    attention_mask=input_ids["attention_mask"].to("cuda"),  \n    do_sample=True,  \n    temperature=1.0,  \n    top_k=50,  \n    top_p=0.9,  \n    max_new_tokens=1024,  \n    eos_token_id=tokenizer.eos_token_id,  \n    pad_token_id=tokenizer.pad_token_id  \n)  \n  \nresponse = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  \nresponse = response.split("### Trả lời:")[1]'

In [2]:
import json

f = open("/root/zaloai/data/math_train.json", "r")
data = json.load(f)

data = data['data']

In [29]:
#from llm_foundry.scripts.train.dataloader import *

def multiple_choice(inp: Dict[str, Union[str, List[str], int]]) -> Dict[str, str]:
    return {
        "prompt" : 'Hãy trả lời bằng một đáp án đúng. Ví dụ: A. Câu hỏi: {query} Lựa chọn: {options}'.format(
            query=inp["question"],
            options=" ".join(inp['choices']),
        ),
        "response" : inp["answer"],
    }

print(multiple_choice(data[0]))
multiple_choice(data[0])['prompt']

{'prompt': 'Hãy trả lời bằng một đáp án đúng. Ví dụ: A. Câu hỏi: Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính Lựa chọn: A. 80,000 : 6 B. 80,000 x 6 C. 80,000 : (6 x 100) D. (80,000 x 6) : 100', 'response': 'D. (80,000 x 6) : 100'}


'Hãy trả lời bằng một đáp án đúng. Ví dụ: A. Câu hỏi: Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính Lựa chọn: A. 80,000 : 6 B. 80,000 x 6 C. 80,000 : (6 x 100) D. (80,000 x 6) : 100'

In [30]:
PROMPT = "### Câu hỏi:\n{instruction}\n\n### Trả lời:"  
  
input_prompt = PROMPT.format_map(  
    {"instruction": multiple_choice(data[0])['prompt']} 
) 

input_ids = tokenizer(input_prompt, return_tensors="pt")  
  
outputs = model.generate(  
    inputs=input_ids["input_ids"].to("cuda"),  
    attention_mask=input_ids["attention_mask"].to("cuda"),  
    do_sample=True,  
    temperature=1.0,  
    top_k=50,  
    top_p=0.9,  
    max_new_tokens=1024,  
    eos_token_id=tokenizer.eos_token_id,  
    pad_token_id=tokenizer.pad_token_id  
)  
  
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  

response = response.split("### Trả lời:")[1]

In [31]:
response

'Để tính số tiền lỗ, ta cần tính lựa chọn sau:\n\nLựa chọn A. 80,000 : 6 B. 80,000 x 6 C. 80,000 : (6 x 100) D. 100\n\nLựa chọn C. 80,000 x 6 D. 80,000 x 6 = 600,000\n\nVậy số tiền lỗ là 600.000 đồng.'

In [ ]:
tokenizer.tokenize(multiple_choice(data[0])['prompt'])


In [10]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '<pad>'}